# 3.1 Gerar Dados para Envio

Este notebook consolida os dados anuais por beneficiario e vinculo familiar para gerar a planilha **DADOS PARA ENVIO**.

**Entradas necessarias:**
- Planilha `dados.xlsx` com abas de `JAN` a `DEZ`
- Em cada aba: `CÓD DA FAMILIA`, `CARTEIRA`, `DEPENDENCIA`, `NOME`, `CPF`, `NASCIMENTO`, `TOTAL`

**Saida:**
- Arquivo `DADOS_PARA_ENVIO.xlsx` com colunas:
  - `CÓD DA FAMILIA`, `CARTEIRA`, `DEPENDENCIA`, `NOME`, `CPF`, `NASCIMENTO`
  - `JAN`, `FEV`, `MAR`, `ABR`, `MAI`, `JUN`, `JUL`, `AGO`, `SET`, `OUT`, `NOV`, `DEZ`
  - `TOTAL`

**Regra de consolidacao:**
- Identificacao por beneficiario:
  - Primeiro por `CPF` (quando existir)
  - Fallback por `NOME + NASCIMENTO`
- Uma linha por **beneficiario + familia** (permite tratar troca de familia no ano).

In [17]:
import pandas as pd
import unicodedata

caminho_dados = '../Data/dados.xlsx'  # Caminho para a planilha anual com abas JAN..DEZ
caminho_saida = '../Data/DADOS_PARA_ENVIO.xlsx'  # Arquivo de saida final

meses = ['JAN 2025', 'FEV 2025', 'MAR 2025', 'ABR 2025', 'MAI 2025', 'JUN 2025', 'JUL 2025', 'AGO 2025', 'SET 2025', 'OUT 2025', 'NOV 2025', 'DEZ 2025']

In [18]:
def normalizar_nome_coluna(nome):
    nome = str(nome).strip().upper()
    nome = ''.join(
        c for c in unicodedata.normalize('NFKD', nome)
        if not unicodedata.combining(c)
    )
    return ' '.join(nome.split())


def mapear_colunas(df, colunas_requeridas):
    mapa_normalizado = {normalizar_nome_coluna(c): c for c in df.columns}
    mapeamento = {}
    faltando = []

    for col in colunas_requeridas:
        chave = normalizar_nome_coluna(col)
        if chave in mapa_normalizado:
            mapeamento[col] = mapa_normalizado[chave]
        else:
            faltando.append(col)

    if faltando:
        raise ValueError(f'Colunas ausentes na planilha: {faltando}')

    return mapeamento


def normalizar_texto(valor):
    if pd.isna(valor):
        return pd.NA
    texto = str(valor).strip()
    if texto == '' or texto.lower() == 'nan':
        return pd.NA
    return ' '.join(texto.split())


def normalizar_cpf(valor):
    if pd.isna(valor):
        return pd.NA
    texto = ''.join(ch for ch in str(valor) if ch.isdigit())
    if texto == '':
        return pd.NA
    return texto


def normalizar_nascimento(valor):
    if pd.isna(valor):
        return pd.NA

    dt = pd.to_datetime(valor, errors='coerce', dayfirst=True)
    if pd.notna(dt):
        return dt.strftime('%Y-%m-%d')

    texto = normalizar_texto(valor)
    return texto


def primeiro_valor_valido(serie):
    serie_valida = serie.dropna()
    if serie_valida.empty:
        return pd.NA
    return serie_valida.iloc[0]


def preparar_mes(df_mes, mes):
    colunas_necessarias = [
        'CÓD DA FAMILIA', 'CARTEIRA', 'DEPENDENCIA',
        'NOME', 'CPF', 'NASCIMENTO', 'TOTAL'
    ]

    mapa = mapear_colunas(df_mes, colunas_necessarias)

    base = pd.DataFrame({
        'CÓD DA FAMILIA': df_mes[mapa['CÓD DA FAMILIA']],
        'CARTEIRA': df_mes[mapa['CARTEIRA']],
        'DEPENDENCIA': df_mes[mapa['DEPENDENCIA']],
        'NOME': df_mes[mapa['NOME']],
        'CPF': df_mes[mapa['CPF']],
        'NASCIMENTO': df_mes[mapa['NASCIMENTO']],
        'TOTAL_MES': df_mes[mapa['TOTAL']],
    }).copy()

    base['NOME'] = base['NOME'].apply(normalizar_texto)
    base['CPF'] = base['CPF'].apply(normalizar_cpf)
    base['NASCIMENTO'] = base['NASCIMENTO'].apply(normalizar_nascimento)
    base['CÓD DA FAMILIA'] = base['CÓD DA FAMILIA'].apply(normalizar_texto)
    base['CARTEIRA'] = base['CARTEIRA'].apply(normalizar_texto)
    base['DEPENDENCIA'] = base['DEPENDENCIA'].apply(normalizar_texto)

    base['TOTAL_MES'] = pd.to_numeric(base['TOTAL_MES'], errors='coerce').fillna(0)

    base = base[base['NOME'].notna()].copy()

    base['CHAVE_BENEFICIARIO'] = base['CPF']
    sem_cpf = base['CHAVE_BENEFICIARIO'].isna()
    base.loc[sem_cpf, 'CHAVE_BENEFICIARIO'] = (
        base.loc[sem_cpf, 'NOME'].fillna('')
        + '|'
        + base.loc[sem_cpf, 'NASCIMENTO'].fillna('SEM_NASCIMENTO')
    )

    base['CHAVE_VINCULO'] = (
        base['CHAVE_BENEFICIARIO'].fillna('SEM_CHAVE')
        + '|FAM:'
        + base['CÓD DA FAMILIA'].fillna('SEM_FAMILIA')
    )

    agrupado = (
        base.groupby(['CHAVE_BENEFICIARIO', 'CHAVE_VINCULO'], as_index=False)
        .agg({
            'CÓD DA FAMILIA': primeiro_valor_valido,
            'CARTEIRA': primeiro_valor_valido,
            'DEPENDENCIA': primeiro_valor_valido,
            'NOME': primeiro_valor_valido,
            'CPF': primeiro_valor_valido,
            'NASCIMENTO': primeiro_valor_valido,
            'TOTAL_MES': 'sum'
        })
    )

    agrupado['MES'] = mes

    return agrupado

In [19]:
registros_mensais = []
meses_lidos = []
meses_com_erro = []

for mes in meses:
    try:
        df_mes = pd.read_excel(caminho_dados, skiprows=1, sheet_name=mes)
        preparado = preparar_mes(df_mes, mes)
        registros_mensais.append(preparado)
        meses_lidos.append(mes)
    except Exception as e:
        meses_com_erro.append((mes, str(e)))

if not registros_mensais:
    raise ValueError('Nenhuma aba mensal foi lida com sucesso. Verifique caminho, abas e colunas da planilha de dados.')

consolidado_mensal = pd.concat(registros_mensais, ignore_index=True)

base_identificacao = (
    consolidado_mensal
    .groupby(['CHAVE_BENEFICIARIO', 'CHAVE_VINCULO'], as_index=False)
    .agg({
        'CÓD DA FAMILIA': primeiro_valor_valido,
        'CARTEIRA': primeiro_valor_valido,
        'DEPENDENCIA': primeiro_valor_valido,
        'NOME': primeiro_valor_valido,
        'CPF': primeiro_valor_valido,
        'NASCIMENTO': primeiro_valor_valido,
    })
)

pivot_meses = (
    consolidado_mensal
    .pivot_table(
        index=['CHAVE_BENEFICIARIO', 'CHAVE_VINCULO'],
        columns='MES',
        values='TOTAL_MES',
        aggfunc='sum',
        fill_value=0
    )
    .reset_index()
)

dados_envio = base_identificacao.merge(
    pivot_meses,
    on=['CHAVE_BENEFICIARIO', 'CHAVE_VINCULO'],
    how='left'
)

for mes in meses:
    if mes not in dados_envio.columns:
        dados_envio[mes] = 0

dados_envio[meses] = dados_envio[meses].apply(pd.to_numeric, errors='coerce').fillna(0)
dados_envio['TOTAL'] = dados_envio[meses].sum(axis=1)

dados_envio_final = (
    dados_envio[
        ['CÓD DA FAMILIA', 'CARTEIRA', 'DEPENDENCIA', 'NOME', 'CPF', 'NASCIMENTO'] + meses + ['TOTAL']
    ]
    .sort_values(['CÓD DA FAMILIA', 'NOME', 'DEPENDENCIA'], na_position='last')
    .reset_index(drop=True)
)

# Insere uma linha vazia entre familias para facilitar visualizacao.
colunas_saida = ['CÓD DA FAMILIA', 'CARTEIRA', 'DEPENDENCIA', 'NOME', 'CPF', 'NASCIMENTO'] + meses + ['TOTAL']
blocos_familia = []

for _, grupo in dados_envio_final.groupby('CÓD DA FAMILIA', dropna=False, sort=False):
    blocos_familia.append(grupo)
    blocos_familia.append(pd.DataFrame([{col: pd.NA for col in colunas_saida}]))

if blocos_familia:
    dados_envio_formatado = pd.concat(blocos_familia, ignore_index=True).iloc[:-1].reset_index(drop=True)
else:
    dados_envio_formatado = dados_envio_final.copy()

display(dados_envio_formatado.head(20))

,CÓD DA FAMILIA,CARTEIRA,DEPENDENCIA,NOME,CPF,NASCIMENTO,JAN 2025,FEV 2025,MAR 2025,ABR 2025,MAI 2025,JUN 2025,JUL 2025,AGO 2025,SET 2025,OUT 2025,NOV 2025,DEZ 2025,TOTAL
0,1.0,1371384000001318.0,Filha,ANA LUIZA GAMA GOUVEIA,13113665639,NaN,372.01,59.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,431.96
1,1.0,1371384000001008.0,Titular,CLAUDIA RIBEIRO GAMA,53223900659,1964-09-25,743.94,803.89,803.89,743.94,803.89,743.94,743.94,863.84,803.89,743.94,743.94,743.94,9286.98
2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,10.0,1371384000010090.0,Esposo,ANTONIO CLARET FRANCO,01617098850,1960-03-31,863.84,743.94,923.79,863.84,743.94,863.84,743.94,803.89,803.89,743.94,863.84,803.89,9766.58
4,10.0,1371384000010007.0,Titular,ELIZABETH ESPAGOLLA FRANCO,50821199668,1962-09-08,798.94,743.94,983.74,743.94,803.89,803.89,803.89,803.89,863.84,743.94,803.89,803.89,9701.68
5,10.0,1371384000010910.0,Agregado,GABRIELA ESPAGOLLA FRANCO,11699641684,1994-02-18,597.76,597.76,597.76,597.76,597.76,597.76,597.76,597.76,597.76,597.76,597.76,597.76,7173.12
6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,100.0,1371384000102009.0,Titular,JOAO CARLOS FERNANDES,47591960644,1958-01-05,707.79,707.79,827.69,767.74,647.84,707.79,707.79,647.84,647.84,767.74,647.84,707.79,8493.48
8,100.0,1371384000100006.0,Titular,JOAO MARTINHO NAKAGAMI,27193705687,1956-08-19,743.94,743.94,743.94,743.94,743.94,743.94,743.94,743.94,743.94,803.89,743.94,803.89,9047.18
9,100.0,1371384000102017.0,Esposa,LEONIDEA DE CARVALHO COUTINHO FERNANDES,35315890678,1960-11-29,647.84,647.84,707.79,707.79,707.79,647.84,707.79,707.79,707.79,707.79,707.79,647.84,8253.68


In [20]:
dados_envio_formatado.to_excel(caminho_saida, index=False)

print(f'Arquivo gerado: {caminho_saida}')
print(f'Meses lidos com sucesso: {meses_lidos}')
print(f'Total de vinculos (sem linha vazia): {len(dados_envio_final)}')
print(f'Total de linhas no arquivo (com linha vazia entre familias): {len(dados_envio_formatado)}')
print(f'Total de beneficiarios unicos: {dados_envio["CHAVE_BENEFICIARIO"].nunique()}')

if meses_com_erro:
    print('\nAvisos de leitura por aba:')
    for mes, erro in meses_com_erro:
        print(f'- {mes}: {erro}')

Arquivo gerado: ../Data/DADOS_PARA_ENVIO.xlsx
Meses lidos com sucesso: ['JAN 2025', 'FEV 2025', 'MAR 2025', 'ABR 2025', 'MAI 2025', 'JUN 2025', 'JUL 2025', 'AGO 2025', 'SET 2025', 'OUT 2025', 'NOV 2025', 'DEZ 2025']
Total de vinculos (sem linha vazia): 1176
Total de linhas no arquivo (com linha vazia entre familias): 1687
Total de beneficiarios unicos: 1175
